# Dash Interactive Viewer Demo

This notebook demonstrates all features of the **Plotly Dash Interactive Viewer** for Trelliscope.

## What You'll Learn

1. ✅ How to create displays with the new `.show_interactive()` method
2. ✅ Filtering by different data types (factor, number, date)
3. ✅ Multi-column sorting with priorities
4. ✅ Layout customization and pagination
5. ✅ Using matplotlib and Plotly panels
6. ✅ Embedding in Jupyter notebooks vs external browser

## Features Overview

| Feature | Description |
|---------|-------------|
| **Filtering** | Multi-select dropdowns, range sliders, date pickers, text search |
| **Sorting** | Multi-column sorting with drag-to-reorder priorities |
| **Pagination** | Navigate through pages with customizable page size |
| **Layout** | Adjust grid columns/rows dynamically (1-6 each) |
| **Labels** | Show metadata beneath each panel |
| **Panels** | PNG images (matplotlib) or interactive Plotly figures |
| **Modes** | External browser, Jupyter inline, or JupyterLab |

In [ ]:
# Setup
import sys
from pathlib import Path

# Add parent directory to path (if running from examples/)
sys.path.insert(0, str(Path.cwd().parent))

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.graph_objects as go

from trelliscope import Display
from trelliscope.meta import FactorMeta, NumberMeta

print("✓ All imports successful")
print(f"  Current directory: {Path.cwd()}")

## 1. Load Sample Data

We'll use the refinery margins dataset with 10 countries and time series data.

In [ ]:
# Load refinery margins data
data_path = Path("_data/refinery_margins.csv")

print(f"Loading data from: {data_path.absolute()}")
df = pd.read_csv(data_path)
df['date'] = pd.to_datetime(df['date'])

print(f"\n✓ Loaded {len(df):,} rows")
print(f"  Shape: {df.shape}")
print(f"  Date range: {df['date'].min().date()} to {df['date'].max().date()}")

# Get countries
countries = sorted(df['country'].unique())
print(f"\nCountries ({len(countries)}): {', '.join(countries[:5])}...")

# Preview
df.head()

## 2. Create Matplotlib Panels

First, let's create static PNG panels using matplotlib.

In [ ]:
def create_matplotlib_plot(country_data, country_name):
    """Create professional matplotlib time series plot."""
    fig, ax = plt.subplots(figsize=(8, 5))
    
    # Plot
    ax.plot(country_data['date'], country_data['refinery_kbd'],
            color='#2c7fb8', linewidth=2, marker='o', markersize=3,
            markerfacecolor='#2c7fb8', markeredgewidth=0)
    
    # Formatting
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
    ax.xaxis.set_major_locator(mdates.YearLocator(2))
    
    ax.set_title(f"Refinery Capacity - {country_name}", 
                 fontsize=12, fontweight='bold')
    ax.set_xlabel("Date", fontsize=10)
    ax.set_ylabel("Refinery (kbd)", fontsize=10)
    ax.grid(True, alpha=0.3, linestyle='--', linewidth=0.5)
    
    plt.tight_layout()
    return fig

# Create data for display
print("Creating matplotlib panels...")
matplotlib_data = []

for country in countries:
    country_df = df[df['country'] == country].copy()
    
    stats = {
        'country': country,
        'avg_capacity': country_df['refinery_kbd'].mean(),
        'max_capacity': country_df['refinery_kbd'].max(),
        'min_capacity': country_df['refinery_kbd'].min(),
        'n_obs': len(country_df)
    }
    
    fig = create_matplotlib_plot(country_df, country)
    matplotlib_data.append({**stats, 'panel': fig})

matplotlib_df = pd.DataFrame(matplotlib_data)
print(f"✓ Created {len(matplotlib_df)} matplotlib panels")
matplotlib_df.head()

## 3. Create Display with Matplotlib Panels

In [ ]:
# Create display
matplotlib_display = Display(
    matplotlib_df,
    name="demo_matplotlib",
    description="Demo: Refinery Capacity with Matplotlib Panels"
)

# Configure metadata
matplotlib_display.set_panel_column("panel")
matplotlib_display.add_meta_variable(
    FactorMeta(varname="country", label="Country", levels=sorted(countries))
)
matplotlib_display.add_meta_variable(
    NumberMeta(varname="avg_capacity", label="Avg Capacity (kbd)", digits=1)
)
matplotlib_display.add_meta_variable(
    NumberMeta(varname="max_capacity", label="Max Capacity (kbd)", digits=1)
)
matplotlib_display.add_meta_variable(
    NumberMeta(varname="min_capacity", label="Min Capacity (kbd)", digits=1)
)
matplotlib_display.add_meta_variable(
    NumberMeta(varname="n_obs", label="# Observations", digits=0)
)

# Set layout and labels
matplotlib_display.set_default_layout(ncol=3, nrow=2, arrangement="row")
matplotlib_display.set_default_labels(["country", "avg_capacity"])

print("✓ Display configured")
matplotlib_display

## 4. Write Display to Disk

In [ ]:
# Write display
output_path = Path("output/demo_matplotlib")
matplotlib_display.write(output_path=output_path, force=True)

# Close figures to free memory
plt.close('all')

print(f"✓ Display written to {output_path}")

## 5. Launch Interactive Dash Viewer 🚀

Now for the exciting part - launch the interactive Dash viewer!

### Option A: External Browser (Default)

Opens in your default browser at http://localhost:8053

**Note**: This will block the cell until you stop it (Ctrl+C or Kernel → Interrupt)

In [ ]:
# Launch in external browser
# matplotlib_display.show_interactive(port=8053)

print("⚠️  Uncomment the line above to launch the viewer")
print("   (It will block this cell until stopped)")

### Option B: Jupyter Inline (Recommended for Notebooks)

Embeds the viewer directly in the notebook!

**Requires**: `pip install jupyter-dash`

In [ ]:
# Launch inline (embedded in notebook)
# matplotlib_display.show_interactive(mode="inline", port=8053)

print("⚠️  Uncomment the line above to embed viewer in notebook")
print("   Requires: pip install jupyter-dash")

## 6. Create Plotly Panels (Interactive)

Now let's create interactive Plotly panels that support hover, zoom, and pan.

In [ ]:
def create_plotly_plot(country_data, country_name):
    """Create interactive Plotly time series plot."""
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=country_data['date'],
        y=country_data['refinery_kbd'],
        mode='lines+markers',
        name='Refinery Capacity',
        line=dict(color='#2c7fb8', width=2),
        marker=dict(size=4, color='#2c7fb8'),
        hovertemplate='<b>Date:</b> %{x|%Y-%m-%d}<br>' +
                      '<b>Capacity:</b> %{y:.1f} kbd<br>' +
                      '<extra></extra>'
    ))
    
    fig.update_layout(
        title=f"Refinery Capacity - {country_name}",
        xaxis_title='Date',
        yaxis_title='Refinery (kbd)',
        plot_bgcolor='white',
        hovermode='closest',
        showlegend=False,
        autosize=True,  # Responsive!
        margin=dict(l=60, r=20, t=40, b=60)
    )
    
    fig.update_xaxes(showgrid=True, gridcolor='#e0e0e0')
    fig.update_yaxes(showgrid=True, gridcolor='#e0e0e0')
    
    return fig

# Create Plotly panels
print("Creating Plotly panels...")
plotly_data = []

for country in countries:
    country_df = df[df['country'] == country].copy()
    
    stats = {
        'country': country,
        'avg_capacity': country_df['refinery_kbd'].mean(),
        'max_capacity': country_df['refinery_kbd'].max(),
        'min_capacity': country_df['refinery_kbd'].min(),
        'n_obs': len(country_df)
    }
    
    fig = create_plotly_plot(country_df, country)
    plotly_data.append({**stats, 'panel': fig})

plotly_df = pd.DataFrame(plotly_data)
print(f"✓ Created {len(plotly_df)} Plotly panels (interactive!)")

## 7. Create Display with Plotly Panels

In [ ]:
# Create display
plotly_display = Display(
    plotly_df,
    name="demo_plotly",
    description="Demo: Refinery Capacity with Interactive Plotly Panels"
)

# Configure (same metadata as matplotlib version)
plotly_display.set_panel_column("panel")
plotly_display.add_meta_variable(
    FactorMeta(varname="country", label="Country", levels=sorted(countries))
)
plotly_display.add_meta_variable(
    NumberMeta(varname="avg_capacity", label="Avg Capacity (kbd)", digits=1)
)
plotly_display.add_meta_variable(
    NumberMeta(varname="max_capacity", label="Max Capacity (kbd)", digits=1)
)
plotly_display.add_meta_variable(
    NumberMeta(varname="min_capacity", label="Min Capacity (kbd)", digits=1)
)
plotly_display.add_meta_variable(
    NumberMeta(varname="n_obs", label="# Observations", digits=0)
)
plotly_display.set_default_layout(ncol=3, nrow=2, arrangement="row")
plotly_display.set_default_labels(["country", "avg_capacity"])

# Write
output_path = Path("output/demo_plotly")
plotly_display.write(output_path=output_path, force=True)

print(f"✓ Display written to {output_path}")

## 8. Launch Plotly Viewer

This viewer will have **interactive Plotly panels** with hover tooltips, zoom, and pan!

In [ ]:
# Launch Plotly version
# plotly_display.show_interactive(port=8054)

print("⚠️  Uncomment the line above to launch the Plotly viewer")
print("   Try hovering over the plots for exact values!")
print("   Try zooming by clicking and dragging!")

## 9. Feature Demonstrations

### What to Try in the Viewer

#### 🔍 Filtering
1. **Country Filter**: Click dropdown → Select 2-3 countries → See panel count update
2. **Capacity Filters**: Drag range sliders → Filter by avg/max/min capacity
3. **Combine Filters**: Use multiple filters together
4. **Clear Filters**: Click "Clear All Filters" button

#### 🔀 Sorting  
1. **Add Sort**: Click "Add Sort" dropdown → Select "Avg Capacity"
2. **Direction**: Click ↑/↓ buttons to toggle ascending/descending
3. **Multiple Sorts**: Add "Country" as secondary sort
4. **Priority**: Note the numbers (1, 2, 3...) showing sort priority
5. **Remove Sort**: Click ✕ button to remove a sort
6. **Clear Sorts**: Click "Clear All Sorts"

#### 📐 Layout
1. **Columns**: Change from 3 to 2, 4, 5... see grid adjust
2. **Rows**: Change from 2 to 1, 3, 4... see page size change
3. **Watch Panels**: Panels resize to fit new layout

#### 📄 Pagination
1. **Navigate**: Click Previous/Next buttons
2. **Page Info**: See "Page N of M" update
3. **Panel Count**: See "Showing X-Y of Z panels"

#### 🏷️ Labels
1. **View Labels**: See Country and Avg Capacity beneath each panel
2. **Formatted Values**: Numbers show 1 decimal place

#### 🎨 Plotly Interaction (Plotly viewer only)
1. **Hover**: Hover over points to see exact values
2. **Zoom**: Click and drag to zoom into a region
3. **Pan**: Click and drag in zoomed view to pan
4. **Reset**: Double-click to reset zoom
5. **Toolbar**: Use Plotly toolbar for download, etc.

## 10. Summary

### What You Learned

✅ **Creating displays** with `.show_interactive()` method  
✅ **Matplotlib panels** for static images  
✅ **Plotly panels** for interactive figures  
✅ **Filtering** by factor and number types  
✅ **Multi-column sorting** with priorities  
✅ **Layout customization** (columns/rows)  
✅ **Pagination** for navigating large datasets  
✅ **Different modes** (external browser vs inline)  

### Key Differences from HTML Viewer

| Feature | HTML Viewer | Dash Viewer |
|---------|-------------|-------------|
| **Technology** | React/Redux (JavaScript) | Dash/Python |
| **Launch** | HTTP server + browser | Python code |
| **Jupyter** | Not embeddable | ✅ Embeddable inline |
| **Plotly Panels** | iframes (slower) | Native (faster) |
| **Customization** | Limited | Full Python control |
| **Sorting** | ✅ Yes | ✅ Yes (new!) |
| **Filtering** | ✅ Yes | ✅ Yes |
| **Views** | ✅ Yes | 🔜 Coming in Phase 3 |

### When to Use Each

**Use HTML Viewer When**:
- Deploying static displays to web servers
- Sharing with non-Python users
- Need maximum compatibility

**Use Dash Viewer When**:
- Working in Jupyter notebooks
- Need Python-based customization
- Want faster Plotly performance
- Prefer native Python tooling

### Next Steps

1. **Try with your own data**: Replace the refinery data with your datasets
2. **Customize plots**: Modify the plot functions for your needs
3. **Add more metadata**: Include additional cognostics for filtering/sorting
4. **Experiment with layouts**: Find the best grid size for your panels
5. **Share with colleagues**: They can use `.show_interactive()` too!

### Resources

- **Documentation**: See `.claude_plans/` folder for detailed specs
- **Test Scripts**: Check `examples/` folder for more examples
- **Issues**: Report bugs or feature requests on GitHub

## Appendix: Installation

```bash
# For basic Dash viewer (external browser)
pip install -e ".[dash]"

# For Jupyter integration (inline mode)
pip install -e ".[jupyter]"

# For everything
pip install -e ".[all]"
```

## Appendix: Troubleshooting

**Viewer doesn't launch**:
- Check that you ran `.write()` first
- Verify port is not in use (try different port)
- Check that dependencies are installed

**Panels don't show**:
- Hard refresh browser (Cmd+Shift+R or Ctrl+Shift+F5)
- Check browser console (F12) for errors
- Verify panel files exist in `output/*/displays/*/panels/`

**Inline mode not working**:
- Install jupyter-dash: `pip install jupyter-dash`
- Restart Jupyter kernel
- Try external mode as fallback

**Slow performance**:
- Reduce panels per page (change rows/columns)
- Use matplotlib instead of Plotly for static panels
- Test with smaller dataset first